## 分析準備：アプリごとの売上基本統計の算出

- 前処理済みのファイル `merged_and_ordered.csv` を読み込む
- 「年月」列を `datetime` 型に変換し、売上列（売上予測（円））を数値型に変換する
- アプリごとに以下の統計量を算出する
  - 観測月数（売上が記録されている月数）
  - 売上合計（総売上）
  - 売上中央値（月次売上の中央値）
  - 最大月売上（最も売上が高かった月）
  - 最終売上月（最後に記録された売上月）
- 結果を `summary_df` として出力し、先頭5行を確認する

### 1.1. データの読み込み

In [27]:
import pandas as pd

# 前処理済みのファイル（merged_and_ordered.csv）を読み込み
file_path = "merged_and_ordered.csv"
df = pd.read_csv(file_path)

### 1.2. 日付と売上列の整形

In [28]:
# 日付・数値変換
df["年月"] = pd.to_datetime(df["年月"], format="%Y/%m", errors="coerce")
df["売上予測（円）"] = pd.to_numeric(df["売上予測（円）"], errors="coerce")

### 1.3. アプリごとの売上統計の算出

In [29]:
def summarize_app(group):
    group = group.sort_values("年月")
    return pd.Series({
        "観測月数": group["売上予測（円）"].count(),
        "売上合計": group["売上予測（円）"].sum(),
        "売上中央値": group["売上予測（円）"].median(),
        "最大月売上": group["売上予測（円）"].max(),
        "最終売上月": group["年月"].max(),
        "サービス終了日": group["サービス終了日"].iloc[0]
    })

# グループキーを手動で指定してから groupby（警告・エラー回避）
summary_df = (
    df.drop(columns="タイトル")
      .groupby(df["タイトル"])
      .apply(summarize_app)
      .reset_index()
)

summary_df["最終売上月"] = summary_df["最終売上月"].dt.strftime("%Y-%m")

### 1.4.　保存と結果表示

In [30]:
summary_df.to_csv("summary_df.csv", index=False)
summary_df.head()

,タイトル,観測月数,売上合計,売上中央値,最大月売上,最終売上月,サービス終了日
0,12オーディンズ - 王道RPG,4,19760000,2605000.0,12850000,2018-04,2018/10/31
1,22/7 音楽の時間,16,533910000,26295000.0,102000000,2021-08,2021/12/22
2,23/7 トゥエンティ スリー セブン,3,45400000,5280000.0,36480000,2018-08,2018/12/27
3,404 GAME RE：SET -エラーゲームリセット-,8,335130000,42960000.0,84530000,2023-11,2024/01/05
4,8 beat Story ~アイドル×音楽ゲーム~,12,13570000,570000.0,4620000,2024-05,2024/06/30
